In [2]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

import julia; julia.install()
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [3]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [10]:
%%julia
using LinearAlgebra;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Generalized Minimum Residual (GMRES)</strong>
</div></div>

# 1. Solving $\mathbf{A x = b}$ as an Optimization Problem

**Problem:** The solution algorithms for $A x = b$ based on GE, QR or the SVD<br>
$\qquad$ require $\mathcal{O}(n^3)$ operations, where $A$ is size $n \times n$.

For large matrices, we need better: Iterative schemes can be designed for different type of matrices.<br>
$\qquad$ the following algoritm tends to work well for large scarse matrices.

## 1.1 Idea

* Convert $A x = b$ into a minimization problem: $x = argmin_{x} \Vert A x -b \Vert$.
* Use a Krylov space: solve for $x$ with the constraint $x \in \mathcal{K}_k$
* Use the Arnoldi algorithm to find an orthonormal basis for $\mathcal{K}_k$

## 1.2 Derivation of the Algorithm

We know the solution(s) of the normal equation satisfy $x = argmin_x \Vert A x - b \Vert.\quad$  (See [20_LengthOrthogonality.ipynb section 3.](20_LengthOrthogonality.ipynb))

Consider the Krylov Space $\quad \mathcal{K}_k(b) = \left( \; b \; A b \; A^2 b \; \dots A^{k-1} b \; \right)$ for a given choice of $k$.<br>
$\qquad$ the Arnoldi Algorithm yields a matrices $Q_m$ with orthonormal columns $q_m$, where $q_1 = \frac{1}{\Vert b \Vert} b$,<br>
$\qquad$ and a matrix $H_k$ such that $A Q_k = Q_{k+1} H_k$. (Note, $H_k$ has size $(k+1) \times k$, i.e.,<br>
$\qquad$ it includes the extra row shown in [the Arnoldi notebook](Arnoldi_Ritz_eigenvalues.ipynb).)

Choosing $x$ in the Krylov space $\mathcal{K}_k$ lets us write $x$ as a linear combination of the columns of $Q_k$, i.e., $x = Q_k \tilde{y}$ for some $\tilde{y}$

The optimization problem becomes<br>
$\qquad\begin{align}
x_k &= argmin_y \Vert\; A Q_k y - \beta q_1 \;\Vert \\
    &= argmin_y \Vert\; Q_{k+1} H_k y - \beta q_1 \;\Vert \\
    &= argmin_y \Vert\; Q_{k+1} \left( H_k y - \beta\ Q^t_{k+1} q_1 \right) \;\Vert \\
    &= argmin_y \Vert\;H_k y - \beta\ Q^t_{k+1} q_1 \;\Vert \\
    &= argmin_y \Vert\;H_k y - \beta\ e_1 \;\Vert
\end{align}$<br>

where $\beta = \Vert b \Vert$, and where we have used $\Vert Q_{k+1} z \Vert = \Vert z \Vert$ for any $z$,<br>
as well as the fact that $Q^t_{k+1} q_1 = \begin{pmatrix} 1 \\ 0 \\ \dots \\ 0 \end{pmatrix} = e_1$ since the $q_i$ are orthonormal.

___
**GMRES:** The least squares solution $x_m$ of $A x - b$ with $x \in \mathcal{K}_k$ is given by

$\qquad  y = argmin_x \Vert H_k y - \beta e_1 \Vert,\quad$ where $\beta=\Vert b \Vert$<br>
$\qquad x_k = Q_k y$

Thus $x_k$ is the $k^{th}$ approximation to the solution of $A x = b$.

# 2. Basic Implementation and Example

Given the problem $A x = b$, where $A \in \mathbb{R}^{n \times n}$.<br>
In exact arithmetic, GMRES gives the exact solution for $k=n$. The goal however is to stop well short, with $k \ll n$.

In [17]:
%%julia
function gmres(A,b,k)
    n        = length(b)
    beta     = norm(b)
    x        = 0
    residual = zeros(k)

    H      = zeros(eltype(A), k+1,k)
    Q      = zeros(eltype(A), n,k+1)
    Q[:,1] = b/beta

    for j in 1:k
        # Next step of Arnoldi iteration.
        v = A*Q[:,j]
        for i in 1:j
            H[i,j] = dot(Q[:,i],v)
            v -= H[i,j]*Q[:,i]
        end
        H[j+1,j] = norm(v)
        Q[:,j+1] = v/H[j+1,j]

        # Solve the minimum residual problem.
        r = [beta; zeros(j)]
        z = H[1:j+1,1:j] \ r
        x = Q[:,1:j]*z
        residual[j] = norm( A*x - b )
    end
    return x, residual
end;

In [57]:
%%julia
function run_gmres(N,s=1)
    A = Tridiagonal( rand(N-1), s*rand(N), rand(N-1))
    x=rand(N)
    b = A*x
    xk,residual =  gmres(A,b,N-1)
end

<PyCall.jlwrap run_gmres>

In [60]:
def show_gmres(matrix_size,main_diag_scale ):
    xk, residual = Main.run_gmres(matrix_size,main_diag_scale)
    return hv.Curve(residual, "k", "residual" )\
             .opts( title="norm(Ax - b) versus k", logy=True, show_grid=True, width=500)
interact( show_gmres, matrix_size=(50,500), main_diag_scale=(0.,20.) )

Column(sizing_mode='fixed')
    [0] Column
        [0] IntSlider(end=500, name='matrix_size', start=50, value=275)
        [1] FloatSlider(end=20.0, name='main_diag_scale', value=10.0)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Curve, height=300, name='interactive01857', sizing_mode='fixed', width=500)

In [13]:
h_exact = hv.Scatter( (np.real(Main.λ), np.imag(Main.λ)), "Re(λ)", "Im(λ)", label="exact" )\
            .opts(size=4, color='black', width=400, height=500, xlim=(-3.5,3.5),ylim=(-3.5,3.5), tools=['hover'])

#def plot_ritz_eigenvals(n):
#    e = Main.ritz_eigenvalues(Main.A, Main.b, n)
#    return (h_exact*hv.Scatter( (np.real(e), np.imag(e)), label="Ritz" ).opts(size=8, title=f"size {n}")).opts(legend_position="top")
#
#interact( plot_ritz_eigenvals, n=(1,Main.N) )

player = pn.widgets.Player(name='Playback', start=1, end=Main.N+1, interval=300)

@pn.depends(value=player.param.value)
def play_ritz_eigenvalues(value):
    e = Main.ritz_eigenvalues(Main.A, Main.b, value)
    return (h_exact*hv.Scatter( (np.real(e), np.imag(e)), label="Ritz" ).opts(size=8, title=f"Exact and Ritz Eigenvalues: size k={value}")).opts(legend_position="top")

pn.Column( player, play_ritz_eigenvalues, width=500)

Column(width=500)
    [0] Player(end=201, interval=300, name='Playback', start=1, value=1, value_throttled=1)
    [1] ParamFunction(function, _pane=HoloViews)

We observe the **usual behaviour** of the algorithm: **the largest eigenvalues $\mathbf{\vert \lambda \vert}$ converge first.**<br>
when $k = n$, the eigenvalues are identical, and we have obtained all of them.

The algorithm is of great interest since it allows us to stop short of $n$: we frequently are interested in only a number of the largest eigenvalues!

To observe the convergence, we compute the distance of the first 6 Ritz eigenvalues to the nearest actual eigenvalue for eack $k = 1, 2, \dots 50$.

In [14]:
%%julia

# experiment with distance of Ritz eigenvalue to actual eigenvalues
# ------------------------------------------------------------------
# KDTree of actual eigenvalues
ev = eigen(A).values
if isreal(ev[1])
    data   = [ev zeros(size(A,1)) ]'
else
    data   = [real(ev) imag(ev) ]'
end

kdtree = KDTree(data);

# Use the KD Tree to compute the shortest distance of a given point to points in the tree.
#k    = 1
#point = rand(2)
#idxs, dists = knn(kdtree, point, k, true)
#@show point
#@show idxs
#@show data[:,idxs]'
#@show dists;

In [15]:
%%julia
"""
given a set of ritz eigenvalues, compute their distance to the nearest actual eigenvalue
"""
function dist_from_nearest_eval( re, k, kdtree)
    err = []
    for p in sort(re, by = x -> abs(x), rev=true)[1:k]
        i, distances = knn(kdtree, [real(p), imag(p)], 1, true)
        push!(err, distances[1])
    end
    err
end;

In [16]:
%%julia
# compute the distance of the first 6 Ritz eigenvalues to the neares actual eigenvalue
N_eigs=6
errors = []
for k in N_eigs:50+N_eigs
   local d = dist_from_nearest_eval( ritz_eigenvalues(A,b,k), N_eigs, kdtree)
   push!(errors, d)
end

In [17]:
h = \
hv.Overlay( [hv.Curve([f[i] for f in Main.errors], "Iteration Number", "distance", label=f"{i}" ).opts(show_grid=True, logy=True) for i in range(Main.N_eigs)] )\
  .opts(legend_position="top", width=500, height=450, title="Error versus Iteration Number")
h

:Overlay
   .Curve.A_0 :Curve   [Iteration Number]   (distance)
   .Curve.A_1 :Curve   [Iteration Number]   (distance)
   .Curve.A_2 :Curve   [Iteration Number]   (distance)
   .Curve.A_3 :Curve   [Iteration Number]   (distance)
   .Curve.A_4 :Curve   [Iteration Number]   (distance)
   .Curve.A_5 :Curve   [Iteration Number]   (distance)

Let us compare the convergence curve to the errors of the power method to compute a dominant eigenvalue:

In [24]:
%%julia
function power_method( A, x, n)
    errs = []
    x = x / norm(x)
    for i in 1:n
        x_old = x
        x     = A * x
        x     = x / norm( x )
        push!( errs, abs( x'A*x - x_old'A*x_old) ) # change in estimated eigenvalue
    end
    errs
end      
errs = power_method( A, b, 100);

In [25]:
# Overlay the Convergence Curves with the change in the dominant eigenvalue estimate
(h*hv.Curve( Main.errs, label="Power Method" )).opts( width=600, legend_position='right', title='Comparison with Power Method Error')

:Overlay
   .Curve.A_0          :Curve   [Iteration Number]   (distance)
   .Curve.A_1          :Curve   [Iteration Number]   (distance)
   .Curve.A_2          :Curve   [Iteration Number]   (distance)
   .Curve.A_3          :Curve   [Iteration Number]   (distance)
   .Curve.A_4          :Curve   [Iteration Number]   (distance)
   .Curve.A_5          :Curve   [Iteration Number]   (distance)
   .Curve.Power_Method :Curve   [x]   (y)

As we see, the Ritz egenvalue estimates converge faster!

Finally, let us investigate the distances of the Ritz eigenvalues to the closest eigenvalues as a function of $k$.

In [20]:
%%julia
function all_distances( A, b, k, kdtree)
    e = ritz_eigenvalues(A, b, k )
    i, distances = knn(kdtree, [real(e) imag(e)]', 1, true) #'

    if length(i) > 0 ret = [i[1] for i in distances] else ret = [] end
    ret
end
#all_distances(A, b, 4, kdtree);

In [21]:
def histogram_of_distances( A, b, k, kdtree, n_bins=20):
    data = Main.all_distances( A, b, k, kdtree )
    sorted_data = np.sort(data)
    h_spikes = hv.Spikes( (range(len(sorted_data)), sorted_data), "n","distance")\
                 .opts(title="Distances from Eigenvalues", alpha=0.5)
    h_scatter = hv.Scatter(sorted_data).opts(size=2, tools=['hover'], show_grid=True)

    frequencies, edges = np.histogram(data, n_bins)
    h_hist = hv.Histogram((edges, frequencies)).opts(title="Distances Histogram")
    return h_spikes*h_scatter + h_hist

interact( lambda k: histogram_of_distances( Main.A, Main.b, k, Main.kdtree, n_bins=20), k=(1,Main.N))

Column
    [0] Column
        [0] IntSlider(end=200, name='k', start=1, value=100)
    [1] Row
        [0] HoloViews(Layout, name='interactive01802')